# EDA Version 0.2
In this approach, we will divide the dataset into 3 Big categories, and then we train each category alone. 


In [13]:
import sys
import os
path = './Data/'
fil = os.path.join(path, 'train.csv')
csvfilename = open(fil, 'r').readlines()
file = 1
splitsize = 200000
for j in range(len(csvfilename)):
    if j % splitsize == 0:
        open(str(fil)+ str(file) + '.csv', 'w+').writelines(csvfilename[j:j+splitsize])
        file += 1

In [7]:
path = './Data'
filepath = os.path.join(path, 'train.csv')
split(open(filepath, 'r'))

AttributeError: '_csv.reader' object has no attribute 'next'

In [1]:
import tensorflow as tf
print(tf.__version__)

KeyboardInterrupt: 

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

## 1- Importing libraries

also doing some logistics

In [ ]:
#Load all the needed libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
import json
from pprint import pprint
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.linear_model import SGDClassifier

%matplotlib inline

In [ ]:
plt.style.use('seaborn-ticks')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

## 2- Loading Train/test data

In [ ]:
#path = '/content/drive/My Drive/Colab Notebooks/ndsc/'
path = './Data'
dftrain = pd.read_csv(os.path.join(path, 'train.csv'))
dftest = pd.read_csv(os.path.join(path, 'test.csv'))


In [ ]:
with open(os.path.join(path,'categories.json')) as f:
    categories = json.load(f)

pprint(categories)

## 3- Feature Engineering

### 3.1 Divide the training data into Big categories containers (Beauty/Fashion/Mobile)

In [ ]:
# 1. add a new col that contins the big category (beauty, fashion, mobile)

newCol = dftrain.image_path.apply(lambda x : x.split('_')[0])
dftrain['mainCat'] = newCol
# dftrain.head()

Check for unique values

In [ ]:
set(dftrain['mainCat'])

### 3.2 Display the total number of samples under each category

In [ ]:
Beautydf= dftrain[dftrain['mainCat'] == 'beauty']
Fashiondf= dftrain[dftrain['mainCat'] == 'fashion']
Mobiledf= dftrain[dftrain['mainCat'] == 'mobile']

plt.bar(['beauty', 'fashion', 'mobile'],[Beautydf.shape[0],Fashiondf.shape[0],Mobiledf.shape[0]],width=0.2, color='black');

#### 3.2.1 EDA

In [ ]:
Fashiondf.head()

In [ ]:
Fashiondf.shape

### 3.3 Randomize the train-dataset

To avoid bias in the training dataset, it is a good practice

In [ ]:
Beautydf.index

In [ ]:
# randomize datasets

Beautydf = Beautydf.reindex(np.random.permutation(Beautydf.index))
Fashiondf = Fashiondf.reindex(np.random.permutation(Fashiondf.index))
Mobiledf = Mobiledf.reindex(np.random.permutation(Mobiledf.index))

# Beautydf.Category.hist(width = 0.5)
# Fashiondf.Category.hist(width = 0.5)
# Mobiledf.Category.hist(width = 0.5)

fig, catAx = plt.subplots(1, 1)

catAx.hist(Beautydf.Category, label='Beauty', width=0.5)
catAx.hist(Fashiondf.Category, label='Fashion', width=0.5)
catAx.hist(Mobiledf.Category , label='Mobile', width=0.5)

catAx.legend()
plt.show()

### 3.4 Remove any number with more than 3 digits in the dataset

For example : `02854323232` which could be an id or something


In [ ]:
# Remove any number in the dataset with more than or equal to 3 digits. 

Beautydf.title.replace(r'\b[0-9]{3,}\b', '', regex=True)
Fashiondf.title.replace(r'\b[0-9]{3,}\b', '', regex=True)
Mobiledf.title.replace(r'\b[0-9]{3,}\b', '', regex=True)

### 3.5 Doing further analysis on the Fashion category

In [ ]:
Fashiondf['Category'].value_counts()

In [ ]:
Fashiondf['Category'].hist();

#### 3.5.1 Check the phrase of the length

In [ ]:
Fashiondf['title_len'] = [len(t) for t in Fashiondf['title']]
Fashiondf['title_wc'] = [len(t.split()) for t in Fashiondf['title']]
Fashiondf.head()

What does the distribution looks like?

In [ ]:
import seaborn as sns
sns.set(style="ticks", palette="pastel")

# Draw a nested boxplot to show bills by day and time
sns.boxplot(x="Category", y="title_len",
 #           hue="smoker", palette=["m", "g"],
            data=Fashiondf)
sns.despine(offset=10, trim=True)

In [ ]:
import seaborn as sns
sns.set(style="ticks", palette="pastel")

# Draw a nested boxplot to show bills by day and time
sns.boxplot(x="Category", y="title_wc",
 #           hue="smoker", palette=["m", "g"],
            data=Fashiondf)
sns.despine(offset=10, trim=True)

Based on the boxplot there's no clear differentiation based on the word count or the length of the title. But there appears to be some outliers on Category 26 Blouse.

### 3.6 Analyzing the count vectors next

In [ ]:
#Fs_vect = CountVectorizer(ngram_range=(1,6), stop_words=stopwords)
cvector = CountVectorizer(ngram_range=(1,6))
cvector.fit(Fashiondf.title)

In [ ]:
len(cvector.get_feature_names())

3,149,454 words have been extracted from the corpus.

In [ ]:
def getTermFrequency(df, category, cat_title):
    '''Given a dataframe and a category, return term frequency'''
    matrix = cvector.transform(df[df.Category == category].title)
    words = matrix.sum(axis=0)
    words_freq = [(word, words[0, idx]) 
                           for word, idx in cvector.vocabulary_.items()]
    tf = pd.DataFrame(list(sorted(words_freq,key = lambda x: x[1], 
                            reverse=True)),columns=['Terms',cat_title])
    tf_df = tf.set_index('Terms')
    return tf_df

### 3.7 Building a word cloud for each category in Fashion

In [ ]:
def generateWordCloud(df, category):
    ''' Generate a word cloud given a dataframe with the respective category'''
    new_df = df[df.Category == category]
    new_words = []
    for t in new_df.title:
        new_words.append(t)
    new_text = pd.Series(new_words).str.cat(sep=' ')
    
    wordcloud = WordCloud(width=1600, height=800, 
                      max_font_size=200).generate(new_text)
    plt.figure(figsize=(12,10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()


In [ ]:
term_freq_dict = {}
for k,v in (categories['Fashion'].items()):
    print ("Generating the word cloud for category {}".format(k))
    generateWordCloud(Fashiondf, v)
    tf = getTermFrequency(Fashiondf, v, k)
    term_freq_dict[v] = tf
    print (tf.head(10))
    print ("\n")

In [ ]:
term_freq_df = pd.concat(term_freq_dict, axis =1)
term_freq_df.head()

#### 3.8 Remove the top common words across all categories which may not be helping with the classification

## 4- Cross Validation

### 4.1 Split the training dataset into train/test

In [ ]:
B_X_train = Beautydf.title
B_y_train = Beautydf.Category

F_X_train = Fashiondf.title
F_y_train = Fashiondf.Category

M_X_train = Mobiledf.title
M_y_train = Mobiledf.Category

In [ ]:
def tokenize_data(vect, X_train, X_test):
    ''' Tokenize the training and test set'''
    # create document-term matrices using the vectorizer
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)

    # print the number of features that were generated
    #print('Features: ', X_train_dtm.shape[1])
    
    return X_train_dtm, X_test_dtm

In [ ]:
skfolds = StratifiedKFold(n_splits=5, random_state=42)
for train_index, test_index in skfolds.split(M_X_train, M_y_train):
    y_train_folds = M_y_train.iloc[train_index]


In [ ]:
import time
from datetime import timedelta
import sklearn.metrics as metrics
import sys

def trainCV(X_train, y_train, vect, folds = 5, random_state=42, 
            clf=SGDClassifier(random_state=42, max_iter=5000, tol=1e-3, 
                              loss='hinge', verbose=1)):
    ''' Train a classifier using CV'''
    #clf = SGDClassifier(random_state=random_state)

    skfolds = StratifiedKFold(n_splits=folds, random_state=random_state)

    cv_start_time = time.monotonic()
    
    scores = []
    features = []
    for train_index, test_index in skfolds.split(X_train, y_train):
        start_time = time.monotonic()
        
        clone_clf = clone(clf)
        #X_train_folds = X_train[train_index]
        y_train_folds = y_train.iloc[train_index]
        #X_test_fold = X_train[test_index]
        y_test_fold = y_train.iloc[test_index]
        
        X_train_folds,X_test_fold = tokenize_data(vect, 
                                                  X_train.iloc[train_index], 
                                                  X_train.iloc[test_index])
        features.append(X_train_folds.shape[1])
        
        clone_clf.fit(X_train_folds, y_train_folds)
        y_pred = clone_clf.predict(X_test_fold)
        n_correct = sum(y_pred == y_test_fold)
        scores.append(n_correct / len(y_pred))
        
        end_time = time.monotonic()
        print("Training time per fold: {}".format(timedelta(seconds=end_time - start_time)))

        
    print ("Features: {}".format(features))
    print("Features variance: %0.2f (+/- %0.2f)" % (np.mean(features), 
                                                    np.std(features) * 2))
    print ("Scores: {}".format(scores))
    print("Accuracy: %0.5f (+/- %0.5f)" % (np.mean(scores), 
                                           np.std(scores) * 2))
    
    cv_end_time = time.monotonic()
    print("Total Training time: {}".format(timedelta(seconds=cv_end_time - cv_start_time)))
    
    return clone_clf, scores, features

### 4.2 Defining Stop words

In [ ]:
# Adding Bahaya Stop words 

factory = StopWordRemoverFactory()
id_stopwords = factory.get_stop_words()
en_stopwords  = list(stopwords.words('english'))
stopwords = id_stopwords + en_stopwords
stopwords[1:20]

#F_stopWords = ['dress', 'lengan', 'wanita', 'women', 'untuk', 'neck', 'model', 'top', 'panjang']
#F_stopWords2 = ['lengan', 'untuk', 'neck', 'model', 'top', 'panjang']
#F_stopWords3 = ['untuk', 'neck', 'model', 'top', 'panjang']
#B_stopWords = ['lip', 'cream', 'promo', 'bb', 'original', 'cushion', 'ml', 'natural', 'nature', 'murah']

### 4.3 Model Training for Beauty

#### 4.3.1 Optiming for nrange

In [ ]:
# finding optimal ngram
means = []
for i in range(2,15):
    vect = CountVectorizer(ngram_range=(1,i))
    print ("Ngram hyperparam: {}".format(i))
    B_sgd, scores, features = trainCV(B_X_train, B_y_train, vect)
    means.append(np.mean(scores))

In [ ]:
label = list(range(2,15))
plt.xlabel('Ngram range')
plt.ylabel('Accuracy')
plt.title('Optimal Nrange param for Beauty')
plt.plot(label, means);

Optimal hyperparam is at nrange 1,8 with Accuracy: 0.78536 (+/- 0.00335)

With slight improvement seen using One vs Rest with Accuracy: 0.78557 (+/- 0.00376)

And with stop words Accuracy: 0.78435 (+/- 0.00327)

#### 4.3.2 Train the optimal model with SGD
Default with max iter = 20, loss = hinge, toll = 1e-3

In [ ]:
ovr_clf = OneVsRestClassifier(SGDClassifier(random_state=42, 
                                            max_iter=5000, 
                                            tol=1e-3,loss='hinge'))
ovr_clf

In [ ]:
B_vect = CountVectorizer(ngram_range=(1,8))
Bs_vect = CountVectorizer(ngram_range=(1,8), stop_words=stopwords)

Using One vs One with stopwords

In [ ]:
B_sgd, B_scores, B_features = trainCV(B_X_train, B_y_train, Bs_vect)

Using one vs rest with stopwords

In [ ]:
Br_sgd, Br_scores, Br_features = trainCV(B_X_train, B_y_train, Bs_vect,
                                         clf=ovr_clf)

Using OvO without stopwords

In [ ]:
Bo_sgd, Bo_scores, Bo_features = trainCV(B_X_train, B_y_train, B_vect)

#### 4.3.3 Using other model for Beauty

Using logistic regression to test

In [ ]:
vect = CountVectorizer(ngram_range=(1,8))
clf = LogisticRegression(random_state=42, solver='lbfgs', 
                         multi_class='multinomial')
B_logr, scores, features = trainCV(B_X_train, B_y_train, vect, clf=clf)

Using multinomial nb

In [ ]:
vect = CountVectorizer(ngram_range=(1,8))
clf = MultinomialNB()
B_nb, scores, features = trainCV(B_X_train, B_y_train, vect, clf=clf)

Using random forest to test

In [ ]:
forest_clf = RandomForestClassifier(random_state=42)
vect = CountVectorizer(ngram_range=(1,8))
B_rf, scores, features = trainCV(B_X_train, B_y_train, vect, clf=forest_clf)

### 4.4 Model Training for Mobile

#### 4.4.1 Optimizing for the best nrange for Mobile

In [ ]:
M_means = []
for i in range(2,15):
    vect = CountVectorizer(ngram_range=(1,i))
    print ("Ngram hyperparam: {}".format(i))
    M_sgd, scores, features = trainCV(M_X_train, M_y_train, vect)
    M_means.append(np.mean(scores))

In [ ]:
label = list(range(2,15))
plt.xlabel('Ngram range')
plt.ylabel('Accuracy')
plt.title('Optimal Nrange param for Mobile')
plt.plot(label, M_means);

Optimal hyperparam for Mobile is with nrange of 1,5 with Accuracy: 0.82828 (+/- 0.00265)

Using One vs Rest is seeing slight improvement with Accuracy: 0.82856 (+/- 0.00143).

And with stop words: Accuracy: 0.82006 (+/- 0.00239).

#### 4.4.2 Training for Optimal Model for Mobile


#### Retrain using One vs rest as it seems to be outperforming One vs One classification

In [ ]:
Ms_vect = CountVectorizer(ngram_range=(1,5), stop_words=stopwords)
M_vect = CountVectorizer(ngram_range=(1,5))

Using OvO with stopwords

In [ ]:
Mo_sgd, Mo_scores, Mo_features = trainCV(M_X_train, M_y_train, Ms_vect)

Using OvR with stopwords

In [ ]:
Mr_sgd, Mr_scores, Mr_features = trainCV(M_X_train, M_y_train, Ms_vect, 
                                        clf=ovr_clf)

Using OvO without stopwords

In [ ]:
M_sgd, M_scores, M_features = trainCV(M_X_train, M_y_train, M_vect)

### 4.5 Model Training for Fashion

#### 4.5.1 Optiming for the nrange for Fashion

In [ ]:
F_vect = CountVectorizer(ngram_range=(1,6), analyzer='word')
Fs_vect = CountVectorizer(ngram_range=(1,6), stop_words=stopwords, analyzer='word')
Fss_vect = CountVectorizer(ngram_range=(1,6), stop_words=stopwords, 
                           analyzer='word', tokenizer=textblob_tokenizer,
                          min=1, max=0.6, max_features=1e5)

F_sgd = OneVsRestClassifier(SGDClassifier(random_state=42, 
                                            max_iter=5000, 
                                            tol=1e-3,
                                            loss='hinge',
                                            alpha=1e-4,
                                            penalty='l2',
                                            verbose =0
                                         ))

In [ ]:
F_means = {}
scores = {}
features = {}
for i in range(1,7):
    for j in range(2,7):
        if i <= j:
            vect = CountVectorizer(ngram_range=(i,j), stop_words=stopwords, analyzer = 'word')
            print ("Ngram hyperparam: {},{}".format(i,j))
            F_sgd, scores[i], features[i] = trainCV(F_X_train, F_y_train, vect, clf=F_sgd)
            F_means["{}_{}".format(i,j)] = np.mean(scores[i])

In [ ]:
k = 0
for i in range(1,7):
    for j in range(2,7):
        if i <= j:
            label = list(range(i,j))
            plt.xlabel('Ngram range')
            plt.ylabel('Accuracy')
            plt.title('Optimal Nrange param for Fashion')
            plt.plot(label, F_means["{}_{}".format(i,j)]);
            k = k + 1

Optimal for Fashion is Ngram range of 1,14 with Accuracy: 0.64612 (+/- 0.00261)

And using One vs Rest with Accuracy: 0.64587 (+/- 0.00192).

With stop words with Accuracy: 0.64569 (+/- 0.00502).

#### 4.5.2 Find Optimal Model for Fashion

Using OvO with stopwords

In [ ]:
Fo_sgd, Fo_scores, Fo_features = trainCV(F_X_train, F_y_train, Fs_vect)

Using OvR with stop words

In [ ]:
Fr_sgd, Fr_scores, Fr_features = trainCV(F_X_train, F_y_train, Fs_vect, 
                                         clf=F_sgd)

Using Ovr with stopwords and stemmer

In [ ]:
Fr_sgd, Fr_scores, Fr_features = trainCV(F_X_train, F_y_train, Fs_vect, 
                                         clf=F_sgd)

Using OvO without stopwords

In [ ]:
F_sgd, F_scores, F_features = trainCV(F_X_train, F_y_train, F_vect, clf=F_sgd)

#### 4.5.3 Manual tuning to optimize SGD

Comparing losses

In [ ]:
def plotScores(func, params, scores):
    '''Plot the scores'''
    plt.title("Effect of {}".format(func))
    plt.xlabel(func)
    plt.ylabel("score")
    x = np.arange(len(params))
    plt.xticks(x, params)
    plt.plot(x, scores)

In [ ]:
# losses
losses = ["hinge", "log", "modified_huber", "perceptron", "squared_hinge"]
l_scores = []

for loss in losses:
    print ("Training using loss: {}".format(loss))
    model = OneVsRestClassifier(SGDClassifier(random_state=42, 
                                            max_iter=5000, 
                                            tol=1e-3,loss=loss))
    F_model, scores, features = trainCV(F_X_train, F_y_train, Fs_vect, 
                                        clf=model)
    l_scores.append(np.mean(scores))

In [ ]:
plotScores("loss", losses, l_scores)

Comparing penalty

In [ ]:
penalty = ["l2", "l1", "none"]
p_scores = []

for p in penalty:
    print ("Training using penalty: {}".format(p))
    model = OneVsRestClassifier(SGDClassifier(random_state=42, 
                                            max_iter=5000, 
                                            tol=1e-3,penalty=p))
    F_model, scores, features = trainCV(F_X_train, F_y_train, Fs_vect, 
                                        clf=model)
    p_scores.append(np.mean(scores))

plotScores("penalty", penalty, p_scores)

In [ ]:
alpha = [0.0001, 0.001, 0.01, 0.1]
a_scores = []

for a in alpha:
    print ("Training using alpha: {}".format(a))
    model = OneVsRestClassifier(SGDClassifier(random_state=42, 
                                            max_iter=5000, 
                                            tol=1e-3,alpha=a))
    F_model, scores, features = trainCV(F_X_train, F_y_train, Fs_vect, 
                                        clf=model)
    a_scores.append(np.mean(scores))

plotScores("alpha", alpha, a_scores)

Using the above param to fine tune the model
* alpha: 0.0001 Accuracy: 0.65286 (+/- 0.00186)
* penalty: l2 Accuracy: 0.65286 (+/- 0.00186)
* loss: hinge Accuracy: 0.65333 (+/- 0.00224)

#### 4.5.4 Optimize the count vectorizer

In [ ]:
for i in np.arange(0.01, 0.1, 0.01):
    print (i)

In [ ]:
Fs_vect

In [ ]:
min_df = [1, 10, 100, 1000, 10000]
min_scores = []

for m in min_df:
    print ("Training using min df: {}".format(m))
    vect = CountVectorizer(ngram_range=(1,6), stop_words=stopwords, analyzer='word',
                                  min_df=m, max_df=1.0, max_features=None)

    F_model, scores, features = trainCV(F_X_train, F_y_train, vect, 
                                        clf=F_sgd)
    min_scores.append(np.mean(scores))



In [ ]:
plotScores("min df", min_df, min_scores)

In [ ]:
max_df = [0.4, 0.6, 0.8, 1.0]
max_scores = []

for m in max_df:
    print ("Training using max df: {}".format(m))
    vect = CountVectorizer(ngram_range=(1,6), stop_words=stopwords, analyzer='word',
                                  min_df=1, max_df=m, max_features=100000)

    F_model, scores, features = trainCV(F_X_train, F_y_train, vect, 
                                        clf=F_sgd)
    max_scores.append(np.mean(scores))



In [ ]:
plotScores("max df", max_df, max_scores)

In [ ]:
max_feat = [10000, 100000, 250000, 500000, None]
mf_scores = []

for m in max_feat:
    print ("Training using max features: {}".format(m))
    vect = CountVectorizer(ngram_range=(1,6), stop_words=stopwords, analyzer='word',
                                  min_df=1e-4, max_df=1.0, max_features=m)

    F_model, scores, features = trainCV(F_X_train, F_y_train, vect, 
                                        clf=F_sgd)
    mf_scores.append(np.mean(scores))

plotScores("max df", max_feat, mf_scores)

Best count vectorizer param appears to be at min df =1, max df = 0.6 and max features at 100,000.

#### 4.5.5 Testing stemming next

In [ ]:
!pip3 install textblob
from textblob import TextBlob

# Use TextBlob
def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

# Use NLTK's PorterStemmer
def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

#### 4.5.4 Grid search to optimize the SGD

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    "loss" : ["hinge", "log", "squared_hinge", "modified_huber"],
    "alpha" : [0.0001, 0.001, 0.01, 0.1],
    "penalty" : ["l2", "l1", "none"],
}

model = SGDClassifier(max_iter=5000, tol=1e-3)
clf = GridSearchCV(model, param_grid=params, cv=5,error_score='raise')

In [ ]:
F_X_train_dtm = tokenize_data(F_vect, F_X_train, F_X_train)
clf.fit(F_X_train_dtm, F_y_train)
print(clf.best_score_)

In [ ]:
print(clf.best_estimator_)

#### 4.5.5 Manual Model

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.base import BaseEstimator

wedding23 = ['wedding','pernikahan','pesta pernikahan', 'gaun']
shirt27 = ['kemeja','shirt','polos']
casual18 = ['motif','musim','musim panas']
maxi20 = ['maxi']
bigsize24 = ['ukuran','ukuran besar']
bodycon22 = ['bodycon','mini', 'sexy']
party19 = ['pesta','party','lace','brokat','brukat']
blouse26 = ['blouse','blus']
tshirt25 = ['kaos', 'kaos shirt', 'pendek', 'lengan pendek']
croptop29 = ['crop','crop top']
tanktop28 = ['tank top','tank', 'top']
others17 = ['baju']
aline21 = ['bahan']
bigsizetop30 = ['jumbo','modal longgar','big size','ada ukuran']

def getFashionCategory(X):
        if any(elem in X.split() for elem in wedding23):
            return 23
        elif any(elem in X.split() for elem in shirt27):
            return 27
        elif any(elem in X.split() for elem in casual18):
            return 18
        elif any(elem in X.split() for elem in maxi20):
            return 20
        elif any(elem in X.split() for elem in bigsize24):
            return 24
        elif any(elem in X.split() for elem in bodycon22):
            return 22
        elif any(elem in X.split() for elem in party19):
            return 19
        elif any(elem in X.split() for elem in blouse26):
            return 26
        elif any(elem in X.split() for elem in tshirt25):
            return 25
        elif any(elem in X.split() for elem in croptop29):
            return 29
        elif any(elem in X.split() for elem in tanktop28):
            return 28
        elif any(elem in X.split() for elem in others17):
            return 17
        elif any(elem in X.split() for elem in aline21):
            return 21
        elif any(elem in X.split() for elem in bigsizetop30):
            return 30
        else:
            return 18 # most common category

class FashionClassifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return X.apply(getFashionCategory)


In [ ]:
F_X_train.head().apply(getFashionCategory)

In [ ]:
from sklearn.model_selection import cross_val_score
F_clf = FashionClassifier()
cross_val_score(F_clf, F_X_train, F_y_train, cv=5, scoring="accuracy")

#### 4.5.6 Other Models

Using Tfid vectorizer

In [ ]:
vect = TfidfVectorizer(ngram_range=(1,6), stop_words=stopwords, analyzer = 'word') #CountVectorizer(ngram_range=(1,14), stop_words=F_stopWords2, analyzer = 'word')
Fsovr_sgd, scores, features = trainCV(F_X_train, F_y_train, vect, clf=F_sgd)

Using Log regression

In [ ]:
clf = LogisticRegression(penalty='l2', max_iter=5000, C=1, solver='lbfgs',
                        multi_class="multinomial")
F_logr, scores, features = trainCV(F_X_train, F_y_train, vect, clf=clf)

Using SVM

In [ ]:
svm_clf = svm.SVC(kernel='rbf') 
F_svm, F_scores, F_features = trainCV(F_X_train, F_y_train, Fs_vect, 
                                         clf=svm_clf)

Using DT

In [ ]:
from sklearn import tree

dt_clf = tree.DecisionTreeClassifier()
F_dt, F_scores, F_features = trainCV(F_X_train, F_y_train, Fs_vect, 
                                         clf=dt_clf)

Using NB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb_clf = MultinomialNB(alpha = 0.07)
F_nb, F_scores, F_features = trainCV(F_X_train, F_y_train, Fs_vect, 
                                         clf=nb_clf)

Using random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features=F_X_train.shape[0], max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)
F_nb, F_scores, F_features = trainCV(F_X_train, F_y_train, Fs_vect, 
                                         clf=rf_clf)

Using xgboost

In [ ]:
from xgboost import XGBClassifier
xg_clf = XGBClassifier(verbosity=2,tree_method='gpu_exact')
F_nb, F_scores, F_features = trainCV(F_X_train, F_y_train, Fs_vect, 
                                         clf=xg_clf)

### 4.6 Checking the scores

#### 4.6.1 Summary:
---
 

1.   **Beauty Category**:    `ngram_range(1, 8) OVR`
---
2.   **Fashion Category**:    `ngram_range(1, 14) OVR`
---
3.   **Mobile Category**:    `ngram_range(1,5) OVR`
---

In [ ]:
total_scores = Bo_scores+ Fo_scores+ Mo_scores
print("OvO with stopwords Accuracy: %0.5f (+/- %0.5f)" % (np.mean(total_scores), 
                                       np.std(total_scores) * 2))

total_scores = B_scores+ F_scores+ M_scores
print("OvO without stopwords Accuracy: %0.5f (+/- %0.5f)" % (np.mean(total_scores), 
                                       np.std(total_scores) * 2))

total_scores = Br_scores+ Fr_scores+ Mr_scores
print("OvR with stopwords Accuracy: %0.5f (+/- %0.5f)" % (np.mean(total_scores), 
                                       np.std(total_scores) * 2))

#### 4.5.1 Result comparison:
* OvO with stopwords Accuracy: 0.75610 (+/- 0.15001)
* OvO without stopwords Accuracy: 0.75598 (+/- 0.15262)
* OvR with stopwords Accuracy: 0.75615 (+/- 0.14967)

Your submission scored 0.76247, which is not an improvement of your best score. Keep trying!


## 5- Modelling

1. Divide the test dataset into 3 Categories
2. Remove numbers more than 3 Digits from the test dataset.
3. Train + Predict.


### 5.1 Divide the test dataset into 3 Categories

In [ ]:
#add category col
newTestCol = dftest.image_path.apply(lambda x: x.split('_')[0])
dftest['newCat'] = newTestCol

In [ ]:
dftest.tail()

In [ ]:
MobileTest = dftest[dftest['newCat'] == 'mobile']
BeautyTest = dftest[dftest['newCat'] == 'beauty']
FashionTest = dftest[dftest['newCat'] == 'fashion']

fig, axt = plt.subplots(1,1)
axt.bar(['beauty', 'fashion', 'mobile'], [len(BeautyTest), len(FashionTest), len(MobileTest)], width=0.5)


### 5.3 Prediction Dataset

In [ ]:
B_X_test = BeautyTest.title
F_X_test = FashionTest.title
M_X_test = MobileTest.title

### 5.4 Prediction Function

In [ ]:
def modelPredict(X_train, y_train, X_test, vect, 
                 clf=SGDClassifier(random_state=42,max_iter=5000, 
                                            tol=1e-3,loss='hinge')):
    ''' Train a classifier using CV'''
    #clf = SGDClassifier(random_state=random_state)
    
    X_train_dtm,X_test_dtm = tokenize_data(vect, X_train, X_test)
    clf.fit(X_train_dtm, y_train)
    y_pred = clf.predict(X_test_dtm)
    return y_pred

In [ ]:
B_y_pred = modelPredict(B_X_train, B_y_train, B_X_test, Bs_vect, clf=ovr_clf)

In [ ]:
F_y_pred = modelPredict(F_X_train, F_y_train, F_X_test, Fs_vect, clf=ovr_clf)

In [ ]:
M_y_pred = modelPredict(M_X_train, M_y_train, M_X_test, Ms_vect, clf=ovr_clf)

### 5.5 Run Prediction

In [ ]:
BeautyTest['Category'] = B_y_pred
FashionTest['Category'] = F_y_pred
MobileTest['Category'] = M_y_pred


## 6- Result

In [ ]:
resDf = BeautyTest.append(FashionTest).append(MobileTest)


In [ ]:
FinalResult = pd.concat([resDf['itemid'], resDf['Category']], axis=1, 
                        keys=['itemid', 'Category'])
FinalResult.to_csv(path + '{}_NDSC_Res.csv'.format(datetime.datetime.today().strftime('%Y%m%d')
                                                  ), index=False)